# M05W03 - Multilayer Perceptrons (MLPs) and Activation Functions 

## Import Packages

In [29]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


## Utils

In [30]:
# Random state set up
random_state = 59
np.random.seed(random_state)
if torch.cuda.is_available():
    torch.cuda.manual_seed(random_state)
else:
    torch.manual_seed(random_state)
    
# Set up Pytorch computational device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Project 1: Auto MPG

### Data Preparation

In [31]:
# Configuration
val_size = 0.2
test_size = 0.125
is_shuffle = True
batch_size = 32

In [32]:
# Implement CustomDataset class to store input data
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.X = x
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [33]:
# Load Data
dataset_dir = 'data'
dataset_filename = 'Auto_MPG_data.csv'
dataset_path = os.path.join('..', dataset_dir, dataset_filename)
dataset = pd.read_csv(dataset_path)

# Data Preprocessing
X = dataset.drop(columns='MPG').to_numpy()
y = dataset['MPG'].to_numpy()

# Train - val - test split
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=val_size,
    random_state=random_state,
    shuffle=is_shuffle
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train,
    test_size=test_size,
    random_state=random_state,
    shuffle=is_shuffle
)

# Standardization
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_val = normalizer.transform(X_val)
X_test = normalizer.transform(X_test)

# Convert to Pytorch tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Initialize CustomDataset obj
train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_val, y_val)
test_dataset = CustomDataset(X_test, y_test)

# Initialize DataLoader
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=5)
val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=5)
test_loader = DataLoader(test_dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        num_workers=5)

### Model Construction

In [ ]:
# Implement MLP structure
class MLP (nn.Module):
    def __init__ (self, input_dims, hidden_dims, output_dims):
        super().__init__()
        self.linear1 = nn.Linear(input_dims, hidden_dims)
        self.linear2 = nn.Linear(hidden_dims, hidden_dims)
        self.output = nn.Linear(hidden_dims, output_dims)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        out = self.output(x)
        return out.squeeze(1)

In [ ]:
# Configuration
input_dims = X_train.shape[1]
output_dims = 1
hidden_dims = 64


# Model intialization
model = MLP(input_dims=input_dims,
            hidden_dims=hidden_dims,
            output_dims=output_dims).to(device)

# Loss Function Selection
criterion = nn.MSELoss()

# Optimizer Selection
momentum = 0.0
weight_decay = o.o
lr = 1e-2
optimizer = torch.optim.SGD(model.parameters(), 
                            momentum=momentum,
                            weight_decay=weight_decay,
                            lr=lr)

# Metric Selection
def r_squared(y_true, y_pred):
    y_true = torch.Tensor(y_true).to(device)
    y_pred = torch.Tensor(y_pred).to(device)
    mean_true = torch.mean(y_true)
    ss_tot = torch.sum((y_true - mean_true) ** 2)
    ss_res = torch.sum((y_true - y_pred) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    return r2